In [1]:
import json, pickle, glob, os, spacy, numpy
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from umap import UMAP
from kiwipiepy import Kiwi, Match
from hdbscan import HDBSCAN
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

/home/jjh/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/jjh/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/jjh/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/jjh/.local/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'num

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
df = pd.read_csv('Titles_2023_Updated.csv')
texts = df['Title'].tolist()
texts = [str(text) for text in texts if isinstance(text, str) and text.strip() != "" and len(str(text).strip()) >= 5]

# Convert 'Publish Date' to datetime format
df['Publish Date'] = pd.to_datetime(df['Publish Date'])
df['YearMonth'] = df['Publish Date'].dt.to_period('M').astype(str)
timestamps = df['YearMonth'].tolist()

In [ ]:
user_stop_word = [
    "이유", "반응", "상황", "해외", "충격", "경악", "공개", "발표", "행동", "긴급",
    "처음", "최고", "외국", "폭발", "폭탄", "역대", "모습", "진짜", "최근", "안녕",
    "반전", "눈물", "소식", "장면", "추천", "생각", "실제", "극찬", "단독", "뜻밖",
    "난리", "속보", "영상", "비상", "정체", "참사", "감동", "당황", "사태", "기적",
    "파격", "소름", "모두", "결국", "근황", "기겁", "특집", "순간", "오열", "초유",
    "의외", "잡식", "한국", "한국인", "초토화", "실시간", "레전드", "끝판왕",
    "트래블튜브", "잡식왕", # channel names
    "TOP", "ft.", "Feat.", "xx"
]

extract_pos_list = ["NNG", "NNP", "SL"]

class CustomTokenizer:
    def __init__(self, kiwi):
        self.kiwi = kiwi

    def __call__(self, text):
        result = []
        for word in self.kiwi.tokenize(text):
            if word[1].startswith('V'):
                modified_word = word[0] + '다'
            else:
                modified_word = word[0]

            if word[1] in extract_pos_list and len(modified_word) > 1 and modified_word not in user_stop_word:
                result.append(modified_word)
        return result

custom_tokenizer = CustomTokenizer(Kiwi())

TypeError: Kiwi.load_user_dictionary() missing 1 required positional argument: 'dict_path'

In [ ]:
# Pre-calculate embeddings
embedding_model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
embeddings = embedding_model.encode(texts, show_progress_bar=True)

Batches:   0%|          | 0/1173 [00:00<?, ?it/s]

In [25]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=False)
vectorizer_model = CountVectorizer(tokenizer=custom_tokenizer)
keybert_model = KeyBERTInspired()
mmr_model = MaximalMarginalRelevance(diversity=0.5)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model
}

In [26]:
model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  #nr_topics=100,
  top_n_words=20,
  calculate_probabilities=False,
  verbose=False
)

# Train model
topics, probs = model.fit_transform(texts, embeddings)

In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

topicdf = model.get_topic_info()
topicdf.drop('MMR', axis=1, inplace=True)
topicdf

,Topic,Count,Name,Representation,KeyBERT,Representative_Docs
0,-1,17427,-1_중국_한국_일본_한국인,"[중국, 한국, 일본, 한국인, 외국인, 세계, 미국, 일본인, 중국인, 유럽, 사진, 나라, 기업, 기술, 잡식, 네티즌, 미국인, 방송, 국가, 언론]","[한국, 최초, 세계, 선진국, 대한민국, 기술, 삼성, 한국인, 일본, 중국]","[중국이 10년 안에 망할 수 밖에 없는 이유, 한국인은 모르지만 전 세계가 부러워 한다는 것, 한국은 일본이야 중국이야? ""일본반응 해외반응""]"
1,0,1943,0_손흥민_토트넘_영국_케인,"[손흥민, 토트넘, 영국, 케인, 감독, 쏘니, 콘테, 현지, 포스테코글루, epl, 리버풀, 경기, 매디슨, 페리시치, 발언, 레비, 전문가, 인터뷰, 히샬리송, 시티]","[토트넘, 손흥민, 맨시티, 아스날, 리버풀, 맨유, 득점왕, 아스널, 잉글랜드, 해트트릭]","[""어떻게 감히 쏘니를...!"" 9번씩이나 당한 토트넘, 난리난 영국 현지 반응, 손흥민 단, 한 장면에 놀란 영국현지 언론에서 경악하는 이유 ""쏘니, 저건 토트넘 식인가요"", ""쏘니가 이럴리가 없어"" 토트넘 팬들이 손흥민 모습 보고 대충격 받은 상황]"
2,1,524,1_폴란드_무기_전차_fa,"[폴란드, 무기, 전차, fa, 독일, 유럽, 국경, 우크라, 방산, 도입, 폴란드군, 국방장관, 루마니아, 육군, kf, 나토, 사격, 바그너, 체코, 전투기]","[폴란드, 폴란드군, 체코, 독일, 우크라, 우크라이나, 러시아, 작심, 감탄, 보도]","[KF-21 무조건 가져오라는 폴란드, [총집편]“한국 혼자 다 했는데요?”韓 천무 사간 폴란드인들마저 ‘당황’, 러시아 폴란드 역대급 상황]"
3,2,480,2_관객_공연_노래_심사,"[관객, 공연, 노래, 심사, 위원, 무대, 아리랑, 군악대, 가수, 연주, 등장, 전통, 이색, 기립박수, 공연장, 독특, 행사, 악기, 한국인, 미국인]","[공연장, 무대, 콘서트장, 공연, 관객, 콘서트, 환호성, 내한, 객석, 행사장]","[실실 비웃던 수천명의 미국 관객과 심사위원들 정확히 10초 후 모두가 뒤집어진 이유, 미국 길거리에서 갑자기 한국인이 등장해 소름돋는 공연을 펼치자 관객들 깜짝!!, 지루해하던 유럽 관객들 뒤집어 놓은 한국인 (공연영상)]"
4,3,465,3_김하성_야구_mlb_메이저리그,"[김하성, 야구, mlb, 메이저리그, 류현진, 홈런, 투수, 선수, 용병, 아내, 샌디에이고, 타자, 메이저, 구단, 관중, 감독, 이정후, 펫코파크, 다저스, 출루]","[메이저리거, 야구장, 김하성, 구단, 배지환, 트레이드, 야구, 투수, 메이저리그, 감독]","[[속보] 김하성 초대박 트레이드 터졌다!!“김하성 가치가 폭발했군요” MLB 발칵초대형 구단서 역대급 제안받은 김하성, [긴급속보] 김하성 초대박! ""MLB역사상 김하성이 처음"", ""김하성 두방이나 터졌다!"" 저 선수 혼자서 야구 다하네요 / 역대급 반응 나오고있는 김하성 상황]"
5,4,424,4_올림픽_도쿄_ioc_선수촌,"[올림픽, 도쿄, ioc, 선수촌, 선수단, 보이콧, 개최, 선수, 취소, 일본, 후쿠시마, 욱일기, 개막식, 폭로, 스가, 침대, 불참, 골판지, 똥물, 최악]","[올림픽, 폐막식, 개막식, 보이콧, 욱일기, 평창, 도쿄, 아베, 일본, 선수촌]","[일본정부에서 도쿄올림픽을 앞두고 큰일을 저질러버린 현재상황, 도쿄올림픽에 한국으로 몰려오는 선수들/한국말이 맞았다, 도쿄 올림픽이 망해도 한국은 떼돈 벌고 있는 충격적인 이유 // ""도대체 일본은 가진 게 뭐야..??""]"
6,5,401,5_독일_독일인_헝가리_여자,"[독일, 독일인, 헝가리, 여자, 친언니, 총리, 친구, 부모, 유럽, 소녀상, 인연, 여동생, 강경화, 남친, 한국, 베를린, 지하철, 자매, 철거, 유럽연합]","[독일인, 독일, 독일어, 메르켈, 체코, 베를린, 나치, 유럽, 남친, 여친]","[독일미녀가 한국에서 충격받은 것, ""독일도 이랬으면 좋겠어요"", 최근 독일에서 공개된 한국 모습 때문에 독일 부모님 난리난 이유.., 독일에선 찾아볼 수 없는데 한국엔 널려있는 이것을 본 독일여자의 충격적인 반응]"
7,6,378,6_러시아_푸틴_우크라이나_우크라,"[러시아, 푸틴, 우크라이나, 우크라, 전쟁, 선언, 침공, 공격, 곰국, 발언, 시작, 제재, 쿠릴열도, 중국, 제안, 반란, 일본, 포고, 뒤통수, 극대노]","[러시아, 러시아인, 푸틴, 러시아군, 우크라이나, 일본해, 쿠릴열도, 독도, 일본, 동해]","[지금 한국때문에 러시아가 난리났어요, ""한국이 러시아를 잡았다!"", 일본이 한국에하듯 똑같이 러시아한테 해봤더니 벌어진일]"
8,7,364,7_배우_헐리웃_여배우_헐리우드,"[배우, 헐리웃, 여배우, 헐리우드, 유명, 시상식, 윤여정, 토크쇼, 스타, 출연, mc, 할리우드, 서울, 톱스타, 진행자, 이정재, 영화, 여배우들, 사랑, 발견]","[헐리웃배우가, 헐리웃배우, 헐리웃, 헐리우드, 여배우들, 여배우, 배우, 할리우드, 출연료, 몸값]","[한국에 빠진 헐리웃유명배우의 한마디에 모두가 한국에 주목, 유명 토크쇼에 나온 헐리웃배우의 발언에 난리난 상황, 한국인과만 일하겠다고 선언해버린 미국 헐리웃 배우의 정체!!ㄷㄷ]"
9,8,362,8_외국인_나라_한국인_선진국,"[외국인, 나라, 한국인, 선진국, 길거리, 존재, 가능, 이사, 이해, 분노, 경험, 치안, 세상, 안전, 한국, 사진, 의식, 사람, 상실, 거짓말]","[배신감, 분노, 비하, 허탈, 한국, 한국인, 무례, 댓글, 일상, 우리나라]","[[단독해외반응] ""한국인들의 정체는 뭐야?"" 한국의 XX문 보고 부러워 미치는 외국인반응 // ""이런 나라에 살아보고 싶다....."", [해외반응] ""한국인들은 매일 이렇게 한다고요?"" 한국의 충격적인 모습에 눈 뒤집혀진 외국인 ""잠시만요... 이게 말이 되는거에요?"", [해외반응] ""여긴 이러면 안되는곳이에요"" 외국인들이 한국인의 행동을 보고 경악한 이유 ""이런건 한국에서만 하세요..""]"


In [ ]:
model.visualize_topics()

In [ ]:
fig = model.visualize_barchart(top_n_topics=20, custom_labels=True)
fig.write_html("BarChart.html")

In [ ]:
hm = model.visualize_heatmap(n_clusters=10, width=1000, height=1000)
hm.write_html("SimilarityMatrix.html")

In [ ]:
doc_viz = model.visualize_documents(texts, custom_labels=True)
doc_viz.write_html("DocumentVisualization.html")

In [ ]:
topic_model = BERTopic().fit(texts)

topic_model.save("SavedModel", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)